<a href="https://colab.research.google.com/github/pekapa/text-based-adventure/blob/main/Text_Based_Adventure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Install Libraries and Imports
# Run this cell first to install necessary libraries and import modules.

# Install the google-genai and ipywidgets libraries
!pip install -q google-genai ipywidgets

import json
import os
import textwrap
import time

from IPython.display import display
import ipywidgets as widgets

import google.genai as genai

# Note: You might need to restart your Colab runtime after installing libraries.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.7 MB/s eta 0:00:00


In [2]:
# @title Set up Google API Key
# Run this cell to set up your Google API Key.
# It attempts to get the key from Colab's user data secrets.
# If not in Colab or the secret is not set, it will print a warning.

# Assuming you are running in a Colab environment and have set up userdata
# If not in Colab, you might need to set the environment variable differently
# For example: os.environ['GOOGLE_API_KEY'] = 'YOUR_ACTUAL_API_KEY'
# Ensure you have the google-genai and ipywidgets libraries installed:
# pip install google-genai ipywidgets
try:
    from google.colab import userdata
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    print("API Key loaded from Colab secrets.")
except ImportError:
    # Fallback for non-Colab environments, assumes GOOGLE_API_KEY is set externally
    if not os.getenv('GOOGLE_API_KEY'):
        print("Warning: GOOGLE_API_KEY environment variable not set. Please set it or use Colab secrets.")
    else:
        print("API Key loaded from environment variable.")

# Choose the model - Using 2.5 Flash for larger context window and tool use
MODEL_NAME = 'gemini-2.5-flash-preview-04-17' # Using the latest 2.5 Flash model


API Key loaded from Colab secrets.


In [3]:
# @title Tool Configuration
# Define the Google Search tool that the AI can use.

# --- Tool Configuration ---
# Define the Google Search tool
# This allows the model to perform searches when it needs external information.

# Define the tool (function declaration) using the dictionary format
google_search_tool = genai.types.FunctionDeclaration(
    name="search_google",
    description="Search Google for information.",
    parameters={
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "The query to search for.",
            }
        },
        "required": ["query"],
    }
)

# List of tools available to the model
AVAILABLE_TOOLS = [google_search_tool]

In [4]:
# @title Global Variables and Constants
# Define global variables for UI elements, game state, and history settings.

# --- Global Variables for UI and Game State ---
game_state = {} # Dictionary to hold the current game state
chat_session = None # Variable to hold the chat session
output_widget = None # ipywidgets.HTML widget for displaying story
input_widget = None # ipywidgets.Text widget for user input
game_ui_box = None # ipywidgets.VBox to hold the entire game UI

# Global variables for pagination and history
turn_states = [] # List to store game_state dictionaries for each turn (most recent at index 0)
current_turn_index = 0 # Index of the currently displayed turn (0 is the latest)
prev_button = None # ipywidgets.Button for navigating to older turns ("Prev" in time)
next_button = None # ipywidgets.Button for navigating to newer turns ("Next" in time)
pagination_label = None # ipywidgets.Label for current page number/memory recollection
pagination_controls = None # ipywidgets.Box for the whole pagination setup

settings = {} # Dictionary to store the finalized settings
settings_finalized = False # Flag to indicate when settings are finalized

# Global variable to track if processing is ongoing
is_processing = False

# Define the maximum number of turns to keep in history
MAX_HISTORY = 50


In [86]:
# @title Helper Functions
# Includes utility functions for the game logic.

# --- Helper Functions ---
def _clear_display_fields(current_state):
    """Helper function to clear the display text fields in a state dictionary."""
    current_state["display_inventory_text"] = ""
    current_state["display_objectives_text"] = ""
    current_state["display_status_effects_text"] = ""
    current_state["display_location_info_text"] = ""
    return current_state


# --- Generic Display UI Initialization ---
def initialize_display_ui(initial_message=""):
    """Initializes and returns the standard game/chat UI widgets."""
    global output_widget, prev_button, next_button, pagination_label, pagination_controls, input_widget, game_ui_box

    output_widget = widgets.HTML(
        value=initial_message,
        layout=widgets.Layout(max_width="130ch", width='auto'), # Use 'ch' for character width
    )
    # Buttons for pagination - adjusted layout for smaller size
    prev_button = widgets.Button(
        description="Prev",
        layout=widgets.Layout(padding='2px 8px') # Reduced padding
    )
    prev_button.disabled = True # Disabled initially as there's no history
    next_button = widgets.Button(
        description="Next",
        layout=widgets.Layout(padding='2px 8px') # Reduced padding
    )
    next_button.disabled = True # Disabled initially as there's no future turns

    pagination_label = widgets.Label(value="") # Initial value will be set by update_display
    input_widget = widgets.Text(placeholder="") # Placeholder set by calling context

    # Arrange widgets in a vertical box
    pagination_controls = widgets.VBox(
        [pagination_label, widgets.HBox([prev_button, next_button])],
        layout=widgets.Layout(
            display='flex',
            flex_flow='column',
            align_items='flex-end', # Align items *within* this VBox to the right
            align_self='flex-end' # Align *this* VBox to the right within its parent
        )
    )


def set_input_state(input_widget, disabled, placeholder=""):
    """Helper function to set the disabled state and placeholder of an input widget."""
    input_widget.disabled = disabled
    input_widget.placeholder = placeholder


def display_message(output_widget, message, is_html=False):
    """Helper function to display a message in a given output widget."""
    with output_widget:
        if is_html:
            display(widgets.HTML(message))
        else:
            print(textwrap.fill(message, width=80))

In [79]:
# @title Proccess responses
# Includes utility functions for the AI response json logic.

def process_ai_response_json(response, previous_state):
    """
    Attempts to parse AI response as JSON and update game state.
    Returns updated game_state dictionary.
    """
    current_state = previous_state.copy() # Start with previous state
    is_tool_call = False # Assume no tool call initially
    response_text = ""

    try:
        # Check if the response contains tool function calls
        if (
            response.candidates and
            response.candidates[0].content.parts and
            response.candidates[0].content.parts[0].function_call
        ):
            is_tool_call = True
            # When a tool call is detected, we don't have narrative/state for this turn yet.
            # Return the previous state but indicate a tool call is happening.
            current_state["narrative"] = "Processing tool output..." # Placeholder narrative
            # Clear display flags as no new status info is available yet
            current_state = _clear_display_fields(current_state)
            return current_state, is_tool_call

        response_text = response.text
        # --- Strip markdown code block markers if present ---
        if response_text.startswith("```json"):
            response_text = response_text[len("```json"):].lstrip()
        if response_text.endswith("```"):
            response_text = response_text[:-len("```")].rstrip()
        # --------------------------------------------------

        full_response_json = json.loads(response_text.strip())

        # Extract narrative and state from the JSON object
        narrative = full_response_json.get("narrative", "Error: Narrative not found in AI response.")
        state_data = full_response_json.get("state", {}) # Default to empty dict if state key is missing

        # Combine narrative and state data into the returned game_state dictionary
        current_state.update({"narrative": narrative, **state_data})

    except json.JSONDecodeError:
        # Print error to standard output for visibility
        print("\n[Warning: Failed to parse AI response as JSON. State might be inconsistent.]")
        print(f"[Raw response: {response_text.strip()[:500]}...]")
        current_state["narrative"] = previous_state.get("narrative", "") + "\n[Parsing Error: Could not understand AI response. Game state may be inconsistent.]"
        current_state = _clear_display_fields(current_state) # Clear display flags on error

    except Exception as e:
        # Print error to standard output for visibility
        print(f"\n[Runtime Error during JSON processing: {e}]")
        print(f"[Raw response: {response_text.strip()[:500]}...]")
        current_state["narrative"] = previous_state.get("narrative", "") + f"\n[An unexpected error occurred: {e}. Game state may be inconsistent.]"
        current_state = _clear_display_fields(current_state) # Clear display flags on error

    return current_state, is_tool_call


def handle_tool_calls(chat_session, previous_state):
    """Handles AI tool calls within a chat session."""
    global is_processing, input_widget

    current_state = previous_state.copy()
    is_tool_call = True # Assume a tool call is happening when this is called

    while is_tool_call:
        # Send an empty message to get the model's response after the tool output
        response = chat_session.send_message("")
        # Process the new response and check if it's still a tool call
        current_state, is_tool_call = process_ai_response_json(response, previous_state=current_state)
        # If the new response is *not* a tool call, the loop exits

    return current_state

In [80]:
# @title Story Display Handlers and Helpers
# Functions to generate HTML for turns and manage the pagination display.

# --- Story display handlers and helpers ---
def generate_turn_html(current_state):
    """Generates HTML content for a single turn from its game_state dictionary."""
    turn_html = ""

    # Add narrative
    narrative = current_state.get("narrative", "").strip()
    if narrative and narrative != "Processing tool output...":
         # Replace newlines with <br> for HTML display
         html_narrative = narrative.replace('\n', '<br>')
         turn_html += f"<p style='font-size: 1.2em;'>{html_narrative}</p>"

    # Add status information if requested
    display_requested = False
    if (
        current_state.get("display_inventory_text", "") or
        current_state.get("display_objectives_text", "") or
        current_state.get("display_status_effects_text", "") or
        current_state.get("display_location_info_text", "")
    ):
        display_requested = True
        turn_html += "<h3>--- Status Information ---</h3>"

    if current_state.get("display_inventory_text", ""):
         turn_html += "<strong>--- Inventory ---</strong><br>"
         inventory_text = current_state["display_inventory_text"].strip().replace('\n', '<br>')
         turn_html += f"<p>{inventory_text}</p>"
    if current_state.get("display_objectives_text", ""):
         turn_html += "<strong>--- Objectives ---</strong><br>"
         objectives_text = current_state["display_objectives_text"].strip().replace('\n', '<br>')
         turn_html += f"<p>{objectives_text}</p>"
    if current_state.get("display_status_effects_text", ""):
         turn_html += "<strong>--- Status Effects ---</strong><br>"
         status_text = current_state["display_status_effects_text"].strip().replace('\n', '<br>')
         turn_html += f"<p>{status_text}</p>"
    if current_state.get("display_location_info_text", ""):
         turn_html += "<strong>--- Location Info ---</strong><br>"
         location_text = current_state["display_location_info_text"].strip().replace('\n', '<br>')
         turn_html += f"<p>{location_text}</p>"

    # If any status was displayed, add a separator for clarity
    if display_requested:
         turn_html += "<hr>" # Use HR tag for a horizontal rule separator

    return turn_html


def update_display():
    """Updates the story display and pagination buttons based on current_turn_index."""
    global output_widget, prev_button, next_button, pagination_label, turn_states, current_turn_index

    if not turn_states:
        output_widget.value = "<p>Game not started yet.</p>"
        prev_button.disabled = True
        next_button.disabled = True
        pagination_label.value = "Memory recollections: 0/0"
        return

    # Generate and display the HTML content for the current turn from the stored state
    current_game_state_to_display = turn_states[current_turn_index]
    output_widget.value = generate_turn_html(current_game_state_to_display)

    # Update button states based on the reversed pagination logic (index 0 is latest)
    prev_button.disabled = (current_turn_index == len(turn_states) - 1) # Disable 'Prev' if at the oldest turn
    next_button.disabled = (current_turn_index == 0) # Disable 'Next' if at the latest turn

    # Update page label
    # Displaying 1-based index for user, and total number of stored turns
    pagination_label.value = f"Memory recollections: {current_turn_index + 1}/{len(turn_states)}"


def on_prev_button_clicked(b):
    """Callback function for the 'Previous' button (navigates to older turns)."""
    global current_turn_index, output_widget, prev_button
    # Check if we can go back further in history (towards higher index)
    if current_turn_index < len(turn_states) - 1:
        current_turn_index += 1 # Move to the next index (older turn)
        update_display()
    else:
        # Display message when trying to go beyond the oldest history
        output_widget.value = "<p><em>The memory of events so long ago is fading... you cannot recall anything further back.</em></p>"
        prev_button.disabled = True # Keep disabled as we are at the start of available history


def on_next_button_clicked(b):
    """Callback function for the 'Next' button (navigates to newer turns)."""
    global current_turn_index, output_widget, next_button
    # Check if we can go forward in history (towards index 0, the latest)
    if current_turn_index > 0:
        current_turn_index -= 1 # Move to the previous index (newer turn)
        update_display()
    else:
        # Display message when trying to go beyond the latest turn
        # This case should ideally not be reached if the button is disabled correctly,
        # but as a fallback or for clarity if the user clicks rapidly.
        output_widget.value = "<p><em>You are looking at current events and don't yet know how to peek into the future.</em></p>"
        next_button.disabled = True # Keep disabled as we are at the latest turn

In [81]:
# @title Pre-Game Settings Chat
# This cell contains the logic for the interactive chat to determine game theme, difficulty, and language using ipywidgets.

def handle_settings_input_submit(sender):
    """Callback function when the user submits input in the settings text box."""
    global chat_session, output_widget, input_widget, settings_finalized, settings, is_processing, game_ui_box, turn_states

    if is_processing: # Prevent multiple submissions while processing
        return

    user_input = sender.value.strip()
    sender.value = "" # Clear the input box after submission

    if not user_input:
        # If input is empty, do nothing and wait for next submission
        return

    is_processing = True # Set processing flag
    current_state = {}
    set_input_state(sender, True, "Processing...") # Disable input and set placeholder

    try:
        # Send user input to the settings chat session
        response = chat_session.send_message(user_input)

        # Use process_ai_response to handle tool calls and initial JSON parsing
        current_state, is_tool_call = process_ai_response_json(response, previous_state=current_state)

        # --- Handle Tool Calls ---
        if is_tool_call:
            # handle_tool_calls will send subsequent messages and update temp_game_state
            handle_tool_calls(chat_session, current_state)
            # Re-enable input after detecting tool call
            is_processing = False
            set_input_state(sender, False, "Tell me more...")
            return # Exit handler


        # Add state to history and update display
        turn_states.insert(0, current_state) # Add to the beginning of history
        turn_states = turn_states[:MAX_HISTORY] # Keep history limited
        current_turn_index = 0 # Show the latest turn
        update_display() # Update the display

        # Check if settings are finalized based on the state
        # Final state has empty narrative and state with settings
        if not current_state.get("narrative"):
            current_state.pop("narrative", None) # Remove narrative if it's empty
            settings = current_state # The remaining attributes should be the settings itself
            if "theme" not in settings:
                raise ValueError("Missing 'theme' in settings")
            if "difficulty" not in settings:
                raise ValueError("Missing 'difficulty' in settings")
            if "language" not in settings:
                raise ValueError("Missing 'language' in settings")

            # Settings are finalized!
            settings_finalized = True # Set the flag

            # # Display finalization message
            # print("-" * 30)
            # print("Settings finalized:")
            # print(f"Theme: {settings['theme']}")
            # print(f"Difficulty: {settings['difficulty']}")
            # print(f"Language: {settings['language']}")
            # print("-" * 30)
            # print("Game setup complete. Starting adventure...")

            # Disable input permanently
            set_input_state(sender, True, "Setup Complete")
            is_processing = False

            # # Hide the settings UI box
            # if game_ui_box:
            #      game_ui_box.layout.display = 'none'

            # --- Trigger the start of the main game ---
            start_main_game(settings)
            # -----------------------------------------

        else:
            # Settings not finalized, continue the settings chat
            is_processing = False
            set_input_state(sender, False, "Tell me more...") # Re-enable input

    except Exception as e:
        # Display error message
        print(f"\n[An unexpected error occurred during settings chat: {e}]")
        is_processing = False
        set_input_state(sender, False, "Enter your command...") # Re-enable input


In [82]:
# @title Main Game Input Handling and Turn Processing
# Contains the core logic for processing user input and AI responses during the main game.

def handle_input_submit(sender):
    """Callback function when the user submits input in the text box."""
    global chat_session, turn_states, current_turn_index, input_widget, is_processing
    current_state = turn_states[0] if turn_states else {}

    if is_processing: # Prevent multiple submissions while processing
        return

    user_input = sender.value.strip()
    sender.value = "" # Clear the input box after submission

    if not user_input:
        # If input is empty, do nothing and wait for next submission
        return

    is_processing = True # Set processing flag
    set_input_state(sender, True, "Generating scene...") # Disable input and set placeholder


    # Check for Python-handled 'quit' command first
    if user_input.lower() == 'quit':
        print("Requesting to exit adventure...") # Print to standard output
        # Modify game_state to trigger the game over condition in the next step
        current_state["game_over"] = True
        # The AI will still process the "Player gives up." input,
        # and the game over check after processing will handle the final state.
        user_input = "Player gives up." # Send this to the AI


    # Process the game turn with the user's input
    # We pass the current game_state so the AI can maintain context
    current_state, is_tool_call = process_ai_response_json(chat_session.send_message(user_input), previous_state=current_state)

    # --- Handle Tool Calls ---
    if is_tool_call:
        set_input_state(sender, True, "[AI is double-checking its information...]")
        current_state = handle_tool_calls(chat_session, current_state)


    # Insert the new state at the beginning of the list (index 0)
    turn_states.insert(0, current_state.copy()) # Store a copy to avoid modification issues

    # Limit the history to MAX_HISTORY turns to avoid memory issues
    turn_states = turn_states[:MAX_HISTORY]
    # The current turn is always the latest, which is now at index 0
    current_turn_index = 0

    # Update the display with the new turn (which is now at index 0)
    update_display()

    is_processing = False # Clear processing flag

    # Check if the game is over based on the processed state
    if game_state.get("game_over", False):
        # Game is over, disable input permanently
        set_input_state(sender, True, "Game Over")
        # Print final game over message to standard output
        print("\n" + "=" * 30)
        print("GAME OVER")
        print("Reason:", game_state.get("game_over_reason", "Unknown."))
        print("=" * 30)
        return # Exit the handler, game is over


    # Re-enable input after processing is complete
    set_input_state(sender, False, "Enter your command...")


In [83]:
# @title System Instructions for initializing the game

def get_settings_instructions():
    return textwrap.dedent("""
        You are a helpful assistant designed to guide the user in selecting a theme and difficulty level for a text adventure game.
        Engage in a natural and friendly conversation with the user.
        Try to keep it short and neutral as this is just the setup stage, ideally not more than 3 iteractions should happen but it's ok to go up to 5 iteractions if needed.
        Start by welcoming them and introducing the text adventure game setup process.
        Ask about their interests, preferred genres, or topics they'd like to learn about.
        Kindly suggest some recent or trending themes as a starting point, do search for recent events that might pose for interesting topics before starting.
        Also, ask about their preferred language for the game (if the user continues in English assume English is fine).
        Once the user seems to have a clear idea or confirms a theme, help them choose a difficulty level (however way the user wants to describe themselve, let them be creative and perhaps suggests a scale based on the selected theme somehow - for example, if the theme is "savanna" the levels could vary from first-time visitor to the lion king of the savannas.).

        You should send the your welcome mesage once you receive the "Start." input from the user.

        Your response for each turn MUST be a single, valid JSON object.
        This JSON object MUST contain two main keys: "narrative" and "state".
        "narrative" should contain the message you want to display the user and will be displayed in an environment that supports a limited set of HTML tags for basic formatting. You can use:
            - `<strong>` for bold text.
            - `<em>` for italic text.
            - `<br>` for line breaks.
            - Use these tags judiciously to enhance readability and highlight important information
        "state" MUST be an empty object while the conversation is ongoing.

        Ensure the entire response for each turn is a single, correctly formatted JSON object with these two top-level keys ("narrative" and "state") i.e.:
        ```json
        {
        "narrative": "The actual text you want displayed to the user",
        "state":{}
        }
        ```

        When the user explicitly states they are satisfied with the chosen theme and difficulty, or indicates they are ready to start the game (e.g., by typing 'done', 'start game', or similar), your *final* response MUST have an empty string for the "narrative" and the game settings for the "state" i.e.:
        ```json
        {
        "narrative": "",
        "state":
            {
            "theme": "Chosen Theme Here",
            "difficulty": "Chosen Difficulty Here",
            "language": "Detected User Language Code (e.g., 'en', 'es', 'fr')"
            }
        }
        ```
        Do NOT include any other text or markdown outside this final JSON object when the user is ready to finish the setup. If the user is not ready, continue the conversation normally.
    """)

def get_game_instruction(theme, difficulty, language):
    """Generates the system instruction string based on user input."""
    return textwrap.dedent(f"""
    You are a dynamic Text Adventure Game Master. Your primary goal is to create an **educational text adventure** for the player.
    The adventure's theme is "{theme}" and the difficulty is set for a "{difficulty}" level player. **The language is set up to: "{language}". All communications with the player must be done in this language**

    **Educational Objective:** Choose a specific, core concept or principle within the theme ("{theme}") that is appropriate for a player at the "{difficulty}" level to learn. Design the entire adventure (narrative, puzzles, clues, characters, interactions) to subtly guide the player towards understanding and eventually deducing this specific concept through gameplay and exploration. The player should learn by doing and observing, not by being directly lectured.

    **Player Interaction:** The player will interact with you using **natural language**. Their input can range from simple commands (e.g., "go north", "examine object") to more elaborate descriptions of their actions, questions about the environment, statements of their understanding or deductions, or attempts to interact with characters or objects in creative ways. Interpret the player's intent from their free-form input and advance the game state and narrative accordingly.

    **Maintaining Consistency:** Always refer to the conversation history, including your previous JSON outputs, to maintain a consistent game state (location, inventory, object status, etc.) and narrative continuity.

    **Output Formatting:** Your narrative and status text will be displayed in an environment that supports a limited set of HTML tags for basic formatting. You can use:
    - `<strong>` for bold text.
    - `<em>` for italic text.
    - `<br>` for line breaks.
    - Use these tags judiciously to enhance readability and highlight important information (e.g., object names, key locations, status effects). Do NOT use complex HTML, just these basic tags.

    **Important:** If the player's input is *primarily* a request to display status information (inventory, objectives, status effects, or location details), set the appropriate `display_..._text` field(s) in the JSON state and provide the requested information in those fields. In such cases, the `narrative` field should contain **minimal or no narrative advancement** – it should acknowledgement of the player's request but also return the state of the narrative. If the player's input includes both an action and a status request, advance the narrative based on the action AND provide the requested status text.

    You have access to a search tool (`search_google`). Use this tool when necessary to find factual information or details related to the theme ("{theme}") or the specific concept you are teaching to make the adventure more accurate, interesting, or to resolve player actions that require external knowledge. You are encouraged to use this tool to set up the adventure as well, especially if the topic is under fast-paced development or inherently involves recent news (such as sports or geopolitics).

    **Game Structure, Pacing & Conclusion:**
    Design the adventure to last approximately **1 hour of gameplay** for a player at the specified difficulty level. This means adjusting the complexity, number of puzzles, and required steps accordingly. The adventure's setting and challenges are dynamically defined by you as we play, consistent with the chosen theme, difficulty, and educational objective.

    The game **ends** and you MUST set the `"game_over"` flag in the JSON state to `true` when any of the following conditions are met:
    1.  The player has successfully demonstrated understanding or deduced the core educational concept.
    2.  The player explicitly states they give up (e.g., by typing "Player gives up.").
    3.  The player encounters a situation that results in their death or a definitive failure from which they cannot recover (e.g., trapped without escape, defeated by an insurmountable obstacle).
    4.  The player has explored all relevant paths or exhausted all meaningful interactions without deducing the concept, and the adventure has reached a natural, non-winning conclusion.

    When the game ends, set `"game_over"` to `true` and provide a brief, accurate `"game_over_reason"` (e.g., "Concept deduced", "Player gave up", "Defeated by guardian", "Reached a dead end").

    Regardless of the `game_over_reason`, provide a final narrative that consolidates the teachings or key takeaways from the adventure so far. **Crucially, the final narrative MUST clearly state the core educational concept** the player was intended to learn. If the player successfully deduced it (Condition 1), explain the real-world concept and explicitly link game events to the principle. For other game-over reasons, summarize what they encountered, highlight relevant clues or principles they interacted with, and explain what the core concept was that the adventure was designed around. **Always end this concluding narrative with a positive, encouraging note to invite the player back for further adventures and continued learning.**

    Your response for each turn MUST be a single, valid JSON object.
    This JSON object MUST contain two main keys: "narrative" and "state".

    The value associated with the "narrative" key should be a string containing the descriptive text for the player. This includes:
    - The current scene and environment.
    - The outcome of the player's action.
    - Any new discoveries, characters, or objects revealed in this turn.
    - Available exits or obvious interactive elements.
    - Keep the tone engaging and consistent with the adventure theme you create. If the game is ending, this will contain the concluding explanation/summary.

    The value associated with the "state" key MUST be another JSON object containing key information about the game state after the player's turn.
    Include the following keys within the "state" object:
    - "location": A brief string describing the player's current location.
    - "inventory": A list of strings representing items the player is currently carrying.
    - "status": A list of strings representing any active status effects or conditions.
    - "objectives": A list of strings representing current goals or quests. Add new objectives as they are revealed. One key objective should relate to understanding the core concept.
    - "exits": a dictionary where keys are directions (e.g., "north", "south", "east", "west", "up", "down") and values are strings describing what lies that way. Include only relevant exits based on the current location. Use HTML tags like `<strong>` for directions.
    - "interactables": a dictionary where the keys represent objects or characters the player can currently interact with in the current location and the values represent their current status (as required for possible interactions and story evolution). Do also include the exits as the player needs to interact with them as well. Use HTML tags like `<em>` for interactable objects.
    - "game_over": A boolean (true/false). This MUST be set to `true` according to the rules defined in the "Game Structure, Pacing & Conclusion" section.
    - "game_over_reason": A string explaining why the game ended (e.g., "Concept deduced", "Player gave up", "Died"). This MUST be set according to the rules defined in the "Game Structure, Pacing & Conclusion" section.
    - "display_inventory_text": A string. If the player's input indicates they want to check their inventory, provide the formatted text describing their current inventory here, using basic HTML tags for formatting. Otherwise, provide an empty string.
    - "display_objectives_text": A string. If the player's input indicates they want to check their current objectives, provide the formatted text listing their objectives here, using basic HTML tags for formatting. Otherwise, provide an empty string.
    - "display_status_effects_text": A string. If the player's input indicates they want to check their current status effects or condition, provide the formatted text describing their status here, using basic HTML tags for formatting. Otherwise, provide an empty string.
    - "display_location_info_text": A string. If the player's input indicates they want a more detailed description of their current surroundings, including exits and interactables, provide the formatted text here, using basic HTML tags for formatting. Otherwise, provide an empty string.


    Ensure the entire response for each turn is a single, correctly formatted JSON object with these two top-level keys ("narrative" and "state").

    Example of a full response (JSON object - Note: The actual game will be in the player's detected language):
    ```json
    {{
      "narrative": "You find yourself in a grand, dusty entrance hall. Sunlight streams through stained-glass windows depicting forgotten heroes. A grand staircase leads north, and a sturdy wooden door is set in the south wall. A marble statue of a stern-looking figure stands in the center.",
      "state": {{
        "location": "Entrance Hall",
        "inventory": [],
        "status": [],
        "objectives": ["Explore the mansion", "Uncover the mansion's secret"],
        "exits": {{"north": "A grand staircase", "south": "A sturdy wooden door"}},
        "interactables": {{"grand staircase": "Nothing is visible beyond it", "sturdy wooden door": "Closed", "marble statue": "Shining nose"}},
        "game_over": false,
        "game_over_reason": "",
        "display_inventory_text": "",
        "display_objectives_text": "",
        "display_status_effects_text": "",
        "display_location_info_text": ""
      }}
    }}
    ```

    You should start when player says "Start.", no matter which language we are actually using. "Player gives up." is also a pre-configured escape route, no matter the configured language.
    Start by describing the initial scene and provide the initial state, based on the theme "{theme}", difficulty "{difficulty}" and language "{language}". Introduce elements that will lead the player toward the core educational concept you've chosen for this session.
    """
    )


In [84]:
# @title Game Initialization and Start
# This cell contains the main `start_adventure` function which sets up the game, initializes the UI, and triggers the first turn. Running this cell will start the game.

def start_main_game(settings):
    """Initializes and starts the main text adventure game with given settings."""
    global output_widget, prev_button, next_button, pagination_label, turn_states, current_turn_index, chat_session, input_widget, game_ui_box, game_state, is_processing, pagination_controls

    # Extract settings
    theme = settings.get("theme", "Generic Adventure")
    difficulty = settings.get("difficulty", "relaxing")
    language = settings.get("language", "en")

    # Generate the system instruction for the main game
    system_instruction = get_game_instruction(theme, difficulty, language)

    # Initialize the client and main game chat session
    client = genai.Client()
    chat_config = genai.types.GenerateContentConfig(
        system_instruction=system_instruction,
        tools=AVAILABLE_TOOLS
    )
    chat_session = client.chats.create( # Reassign global chat_session for main game
        model=MODEL_NAME,
        config=chat_config,
        history=[] # Start with empty history for the main game
    )

    # Reset main game state and history
    turn_states = []
    game_state = {}
    is_processing = False # Ensure processing flag is reset for main game

    # Re-arrange the main game UI box to include pagination
    # game_ui_box was created and displayed in start_adventure
    game_ui_box.children = [pagination_controls, output_widget, input_widget]

    # # Ensure the main game UI box is visible (it was hidden after settings finalized)
    # game_ui_box.layout.display = 'flex' # Or 'block'

    # Relink button clicks to main game handlers (redundant but safe)
    prev_button.on_click(on_prev_button_clicked)
    next_button.on_click(on_next_button_clicked)

    # Link input text submission to the main game handler
    input_widget.on_submit(handle_input_submit)

    # Trigger the first turn of the main game
    set_input_state(input_widget, False, "Enter your command...") # Ensure input is enabled
    input_widget.value = "Start."
    handle_input_submit(input_widget) # Simulate initial input


def setup_adventure():
    """Initializes and starts the text adventure game setup process."""
    global output_widget, input_widget, game_ui_box, chat_session, settings, settings_finalized, is_processing, prev_button, next_button, pagination_label, pagination_controls

    is_processing = False
    settings_finalized = False # Ensure this is reset at the start of a new adventure setup
    settings = {} # Ensure this is reset

    try:
        # Set up ipywidgets for the main game UI using the generic function
        # This initializes output_widget, input_widget, pagination_controls, etc.
        # The initial message is for the setup phase
        initialize_display_ui(initial_message="Loading setup chat...")

        # Create the AI Client and start a chat session for settings
        client = genai.Client()

        chat_session = client.chats.create( # Use chat_session for settings chat initially
            model=MODEL_NAME,
            history=[],
            config=genai.types.GenerateContentConfig(
                system_instruction=get_settings_instructions(),
                tools=AVAILABLE_TOOLS,
            ),
        )

        # Link input text submission to the SETTINGS handler function
        input_widget.on_submit(handle_settings_input_submit)

        # The main box layout adjusted for the pre-chat - only show output and input
        # We need to create game_ui_box here to display it
        game_ui_box = widgets.VBox([output_widget, input_widget])


        # Display the entire game UI (initially just the settings chat part)
        display(game_ui_box)

        # Trigger the initial AI welcome message in the settings chat
        # Send an empty message to get the AI's first turn based on the system instruction
        input_widget.value = "Start."
        handle_settings_input_submit(input_widget) # Simulate initial input to trigger AI response

        # The start_adventure function now finishes here.
        # The transition to the main game is handled by handle_settings_input_submit
        # when settings_finalized becomes True.

    except Exception as e:
        print(f"\nAn error occurred during game setup: {e}")
        print("Please ensure your API key is correctly set and the model name is valid.")

In [85]:
# @title Execute the Game
# Run this cell to start the adventure!

if __name__ == "__main__":
    setup_adventure()
